# Demo of functions ds\_focus from*./swiss_utils/data_cube_utilities/sdc_devtools.py*

*****

__This script is the "official demo" of a function. Please if you want to modify it, work on your own copy__

In order to ease the creation of demo dataset to be used during development, the `ds_focus` function selects within an xarray.Dataset (`ds`) the AOI (size given by `dough_width`) with sum of min, max values (defined by `stat`argument). Then focus can be automatically given on a small datset with maximum or minimum of data.

__ds_focus__:
- **ds**: xarray.Dataset to to be focused on.
- **dough_width** : doughnut width (window size will be 2 * dough_size + 1).
- **stat** : stat type to apply for window selection.

Documentation for a given function can be accessed simply by adding ? at the end of the function in a cell. e.g. `ds_focus?` or by selecting the function and pressing `Shift-Tab`.

In this demo Jupyter script, the user can either use the in-script function (below) or import it from ./swiss_utils/data_cube_utilities/sdc_devtools.py.

In [ ]:
# Make sure the script is using the proper kernel
try:
    %run ../swiss_utils/assert_env.py
except:
    %run ./swiss_utils/assert_env.py

In [ ]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py

# to plot figures
%matplotlib inline

import numpy as np
import xarray as xr

from datetime import datetime
from matplotlib import colors

from swiss_utils.data_cube_utilities.sdc_utilities import load_multi_clean
from swiss_utils.data_cube_utilities.sdc_advutils import draw_map, oneband_fig

import datacube
dc = datacube.Datacube()

# AND THE FUNCTION
from swiss_utils.data_cube_utilities.sdc_devtools import ds_focus

### First create a large, possibly with dataset with Landsat 7 as this product is know to contains plenty of nodata since 2003.

The next cell contains the dataset configuration information:
- product
- geographical extent
- time period
- bands

You can generate it in three ways:
1. manually from scratch,
2. by manually copy/pasting the final cell content of the [config_tool](config_tool.ipynb) notebook,
3. by loading the final cell content of the [config_tool](config_tool.ipynb) notebook using the magic `# %load config_cell.txt`.

In [ ]:
%load config_cell.txt

In [ ]:
draw_map([min_lat, max_lat], [min_lon, max_lon], draw = False)[0]

In [ ]:
# Load the data and clean it
dataset_clean, clean_mask = load_multi_clean(dc = dc,
                                             products = product ,
                                             time = [start_date, end_date],
                                             lon = (min_lon, max_lon),
                                             lat = (min_lat, max_lat),
                                             measurements = measurements)

### As can be seen in the next cell the number of data through time differ greatly between pixels

In [ ]:
dataset_count = dataset_clean[measurements[0]].count(dim=['time'])
oneband_fig(dataset_count,
            leg = colors.LinearSegmentedColormap.from_list('redtogreen', ['darkred', 'darkgreen'], N=256),
            title = 'Data count',
            scalebar_color = 'white',
            max_size = 6)

### Let's pretend, as a developer you are interest to start writing your code with location containing as much data as possible (green pixels in the figure above)

In [ ]:
# a dough_width (dough for doughnut) value of 25 will create an AOI of 51 x 51 pixels (2 * 25 + 1)
ds_dev = ds_focus(ds = dataset_clean, dough_width = 25, stat = 'max')

In [ ]:
# Let's visualize the development dataset

dataset_count = ds_dev[measurements[0]].count(dim=['time'])
oneband_fig(dataset_count,
            leg = colors.LinearSegmentedColormap.from_list('redtogreen', ['darkred', 'darkgreen'], N=256),
            title = 'Data count',
            scalebar_color = 'white',
            max_size = 6)

draw_map(lat_ext = (ds_dev.latitude.values.min(), ds_dev.latitude.values.max()),
         lon_ext = (ds_dev.longitude.values.min(), ds_dev.longitude.values.max()),
         draw = False)[0]

### Once main code written, lets focus on location containing as few data as possible (red pixels in the figure above)

In [ ]:
# simply change the stat argument to 'min'
ds_dev = ds_focus(ds = dataset_clean, dough_width = 25, stat = 'min')

In [ ]:
# Let's visualize the new development dataset

dataset_count = ds_dev[measurements[0]].count(dim=['time'])
oneband_fig(dataset_count,
            leg = colors.LinearSegmentedColormap.from_list('redtogreen', ['darkred', 'darkgreen'], N=256),
            title = 'Data count',
            scalebar_color = 'white',
            max_size = 6)

draw_map(lat_ext = (ds_dev.latitude.values.min(), ds_dev.latitude.values.max()),
         lon_ext = (ds_dev.longitude.values.min(), ds_dev.longitude.values.max()),
         draw = False)[0]